In [35]:
from datasets import load_dataset

dataset64 = load_dataset("json", data_files="./comments_guns_with_scores_above_quantile_0.90_train_64.json")

Found cached dataset json (C:/Users/ysong/.cache/huggingface/datasets/json/default-7be31e0ae052b38a/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)
100%|██████████| 1/1 [00:00<00:00, 334.90it/s]


In [36]:
dataset64 = dataset64["train"]

In [37]:
dataset64["subreddit"][10:30]

['progun',
 'GunsAreCool',
 'progun',
 'GunsAreCool',
 'progun',
 'GunsAreCool',
 'progun',
 'progun',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'progun',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'GunsAreCool',
 'progun']

In [38]:
dataset64[0]

{'link_id': 't3_aud8x',
 'subreddit': 'guncontrol',
 'retrieved_on': 1426196154.0,
 'score': 6.0,
 'body': "Nothing. I was just pointing that out to demonstrate that, although Jesse Ventura is cool, he's not right about everything.",
 'gilded': 0,
 'id': 'c0jgzwt',
 'predicted_community': 3,
 'subreddit_id': 't5_2rdus',
 'created_utc': 1264557266,
 'controversiality': 0,
 'edited': -1.0,
 'parent_id': 't1_c0jgqpd',
 'language': 'en',
 'distinguished': None,
 'body_cleaned': "nothing . i was just pointing that out to demonstrate that , although jesse ventura is cool , he 's not right about everything .",
 'author': 'HAZod'}

In [39]:
import pandas as pd
df = pd.DataFrame(dataset64)
subreddit_counts = df['subreddit'].value_counts()
subreddit_counts

subreddit
GunsAreCool              2699
liberalgunowners         1307
progun                   1144
gunpolitics               321
guncontrol                112
shitguncontrollerssay      49
2ALiberals                 44
neveragainmovement         25
nra                        10
NOWTTYG                     4
secondamendment             2
Name: count, dtype: int64

In [40]:
predicted_community = df["predicted_community"].value_counts()
predicted_community

predicted_community
3    2815
2    1526
0    1376
Name: count, dtype: int64

In [34]:
148 / (148 + 654 + 911)

0.08639813193228255

## Top sentencens

In [14]:
filtered_df = df[df['subreddit'].isin(['progun', 'liberalgunowners', 'guncontrol'])]

In [18]:
sorted_df = filtered_df.sort_values('score', ascending=False)
top_10_sentences = sorted_df.head(10)[['body', 'subreddit']]

In [19]:
for index, row in top_10_sentences.iterrows():
    print("Subreddit:", row['subreddit'])
    print("Sentence:", row['body'])
    print()



Subreddit: progun
Sentence: As a federal LEO, the very idea of confiscating guns is laughable. I swore an oath to the constitution, not to Beto or any other politician.

Subreddit: progun
Sentence: We must ban this horrible weapon of war

Subreddit: liberalgunowners
Sentence: This is also why I love Hickok45. He's just an all-around good guy. One of the best representatives out there for the pro-2A community.

Subreddit: liberalgunowners
Sentence: for good reason.


god she sucked so much

Subreddit: liberalgunowners
Sentence: He is the worst parts of a Democrat and the worst parts of a Republican, rolled into one. An authoritarian, through and through.

Subreddit: progun
Sentence: "You're pretty stupid to not just have armed security like me you filthy peasants." That's all I read.

Subreddit: progun
Sentence: Meanwhile in England, an old man is being charged with murder after he killed one of the men breaking into his home

Subreddit: liberalgunowners
Sentence: This is part of why he